# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
nltk.download('punkt')

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tcanty\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tcanty\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///Messages.db')

In [3]:
engine.table_names()

['Messages']

In [4]:
df = pd.read_sql('SELECT * FROM "Messages"', engine)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26215 entries, 0 to 26214
Data columns (total 39 columns):
message                   26215 non-null object
original                  10170 non-null object
genre                     26215 non-null object
related                   26215 non-null int64
request                   26215 non-null int64
offer                     26215 non-null int64
aid_related               26215 non-null int64
medical_help              26215 non-null int64
medical_products          26215 non-null int64
search_and_rescue         26215 non-null int64
security                  26215 non-null int64
military                  26215 non-null int64
child_alone               26215 non-null int64
water                     26215 non-null int64
food                      26215 non-null int64
shelter                   26215 non-null int64
clothing                  26215 non-null int64
money                     26215 non-null int64
missing_people            26215 non-null i

In [5]:
X = df[['message','genre']]
Y = df.drop(columns=['message','original','genre'])

In [6]:
X = X.merge(pd.get_dummies(X['genre']),left_index=True, right_index=True).drop(columns=['genre'])

In [7]:
X['message'] = X['message'].str.lower()

In [8]:
X.head()

,message,direct,news,social
0,weather update - a cold front from cuba that c...,1,0,0
1,is the hurricane over or is it not over,1,0,0
2,looking for someone but no name,1,0,0
3,un reports leogane 80-90 destroyed. only hospi...,1,0,0
4,"says: west side of haiti, rest of the country ...",1,0,0


In [9]:
X['message'] = X['message'].apply(lambda x: re.sub('[^\w\s]','',x))

### 2. Write a tokenization function to process your text data

In [10]:
def tokenize(text):
    tokens = word_tokenize(re.sub('[^\w\s]',' ',text))
    lem = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lem.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
       
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [11]:

pipeline = Pipeline([
    ('text_pipeline', Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer())
    ])),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
    

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [12]:
X_train, X_test, Y_train, Y_test = train_test_split(X['message'],Y,test_size=0.3,random_state = 208)
pipeline.fit(X_train,Y_train)

C:\Users\tcanty\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\tcanty\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\tcanty\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\tcanty\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in

Pipeline(memory=None,
     steps=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:
Y_predict = pipeline.predict(X_test)

In [14]:
Y_test.values[:,0].shape

(7865,)

In [15]:
for i in range(Y_test.shape[1]):
    print(Y_test.columns[i])
    print(classification_report(Y_predict[:,i],Y_test.values[:,i]))

related
              precision    recall  f1-score   support

           0       0.34      0.60      0.43      1032
           1       0.93      0.82      0.87      6774
           2       0.32      0.34      0.33        59

   micro avg       0.78      0.78      0.78      7865
   macro avg       0.53      0.59      0.54      7865
weighted avg       0.85      0.78      0.81      7865

request
              precision    recall  f1-score   support

           0       0.98      0.88      0.93      7239
           1       0.37      0.82      0.51       626

   micro avg       0.88      0.88      0.88      7865
   macro avg       0.68      0.85      0.72      7865
weighted avg       0.93      0.88      0.90      7865

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7864
           1       0.00      0.00      0.00         1

   micro avg       1.00      1.00      1.00      7865
   macro avg       0.50      0.50      0.50      786

C:\Users\tcanty\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [16]:
pipeline.steps

[('text_pipeline', Pipeline(memory=None,
       steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=None, min_df=1,
          ngram_range=(1, 1), preprocessor=None, stop_words=None,
          strip...y=None)), ('tfidf', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True))])),
 ('clf',
  MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
              max_depth=None, max_features='auto', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
              oob_score=False, random_state=None, verbose=0,
              warm_start=False),
             n_jobs=None))]

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {'max_depth':[5, 10, 20, 40, 60],
             }

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.